In [1]:
from zipfile import ZipFile

In [2]:
#Extract the zip file of the initial cleaned data
file_name = "Cleaned_data_v2.zip"
with ZipFile(file_name, 'r') as zip:
    zip.printdir()
    zip.extractall()

File Name                                             Modified             Size
Cleaned_data_v2/cleaned_car_data_v2.csv        2023-03-14 14:32:22    322683285


In [3]:
#Import used
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

In [4]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.12.0-rc1


In [5]:
#Convert data to Dataframe
dataset = pd.read_csv("Cleaned_data_v2/cleaned_car_data_v2.csv", sep=',')
test_results = {}
dataset

C:\Users\colec\AppData\Local\Temp\ipykernel_13408\621276206.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("Cleaned_data_v2/cleaned_car_data_v2.csv", sep=',')


,price,body_type,exterior_color,has_accidents,listed_date,make_name,mileage,model_name,owner_count,year,dealer_zip,major_options,trim_name,wheel_system
0,200.0,SUV / Crossover,BLUE,False,2020-07-07,Ford,150000.0,Explorer,4.0,2005,33178,[3],XLT V6,RWD
1,249.0,Sedan,Cashmere Metallic,True,2020-08-26,Buick,202158.0,Century,5.0,2005,49684,[3],Custom Sedan FWD,FWD
2,347.0,Sedan,Titanium Gray Metallic,False,2020-07-16,Mazda,213512.0,MAZDA3,1.0,2006,84107,[3],i,FWD
3,395.0,SUV / Crossover,Black,False,2020-05-02,Jeep,122610.0,Liberty,3.0,2002,44256,[3],Sport 4WD,4WD
4,400.0,Sedan,SILVER,True,2016-07-26,Oldsmobile,111111.0,Alero,1.0,2002,29379,[3],GX,FWD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528603,2175000.0,SUV / Crossover,Silver,False,2020-06-28,Nissan,570.0,Rogue Sport,1.0,2020,67801,"[3, 4, 8, 9, 10]",S FWD,FWD
2528604,2250000.0,Sedan,Black,False,2020-05-14,Chevrolet,7852.0,Impala,1.0,2020,67801,"[9, 3, 4, 6]",LT FWD,FWD
2528605,2698500.0,SUV / Crossover,White,False,2020-09-02,Ford,39060.0,Edge,1.0,2018,4096,"[8, 9, 3, 4]",SEL AWD,AWD
2528606,2738900.0,Coupe,Nero,False,2020-02-01,Ferrari,221.0,LaFerrari,2.0,2015,90211,"[8, 4]",Coupe,RWD


In [6]:
def one_hot(x):
    x = x.strip("[] ").split(",")
    #print(x)
    temp = [0] * 77
    for a in x:
        temp[int(a)] = 1
    #print(temp)
    return temp

one_hot_col_lst = []
for a in range(77):
    one_hot_col_lst.append(str(a))


In [7]:
#Sets accidents to either 0 or 1 depending on if the column value was True or False
dataset['has_accidents'] = dataset['has_accidents'].astype(bool).astype(int)
#One hot encodes each of the following columns 
dataset = pd.get_dummies(dataset, columns=['wheel_system'], prefix='', prefix_sep='')
dataset = pd.get_dummies(dataset, columns=['make_name'], prefix='', prefix_sep='')

dataset['major_options'] = dataset['major_options'].apply(one_hot)
dataset[one_hot_col_lst] =pd.DataFrame(dataset.pop('major_options').values.tolist(), index=dataset.index)

name_dict= {}
i =1000
def names(x):
    global i
    out = ""
    for a in x:
        if a in name_dict:
            out+=str(name_dict[a])
        else:
            name_dict[a]=i
            out+=str(i)
            i+=1
    return int(out)

dataset['model_name'] = dataset.apply(lambda row: row['model_name'].replace(" ", ""), axis=1)
dataset['body_type'] = dataset.apply(lambda row: row['body_type'].replace(" ", ""), axis=1)
dataset['model_name'] = dataset['model_name']+ " "+ dataset['body_type']
dataset['model_name'] = dataset['model_name'].str.split(" ")
dataset['unique_number'] = dataset['model_name'].apply(names)


dataset = pd.get_dummies(dataset, columns=['body_type'], prefix='', prefix_sep='')

#Color can eventually be included, but would prefer them to be more simplfied
dataset = dataset.drop(columns=['exterior_color', 'model_name', 'listed_date', 'dealer_zip', 'trim_name'])
dataset

,price,has_accidents,mileage,owner_count,year,4WD,4X2,AWD,FWD,RWD,...,unique_number,Convertible,Coupe,Hatchback,Minivan,PickupTruck,SUV/Crossover,Sedan,Van,Wagon
0,200.0,0,150000.0,4.0,2005,0,0,0,0,1,...,10001001,0,0,0,0,0,1,0,0,0
1,249.0,1,202158.0,5.0,2005,0,0,0,1,0,...,10021003,0,0,0,0,0,0,1,0,0
2,347.0,0,213512.0,1.0,2006,0,0,0,1,0,...,10041003,0,0,0,0,0,0,1,0,0
3,395.0,0,122610.0,3.0,2002,1,0,0,0,0,...,10051001,0,0,0,0,0,1,0,0,0
4,400.0,1,111111.0,1.0,2002,0,0,0,1,0,...,10061003,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528603,2175000.0,0,570.0,1.0,2020,0,0,0,1,0,...,17511001,0,0,0,0,0,1,0,0,0
2528604,2250000.0,0,7852.0,1.0,2020,0,0,0,1,0,...,10931003,0,0,0,0,0,0,1,0,0
2528605,2698500.0,0,39060.0,1.0,2018,0,0,1,0,0,...,13021001,0,0,0,0,0,1,0,0,0
2528606,2738900.0,0,221.0,2.0,2015,0,0,0,0,1,...,20741050,0,1,0,0,0,0,0,0,0


In [8]:
above15k = dataset[dataset['mileage'] >= 15000]
remaining = dataset[dataset['mileage'] < 15000]

a15k_train_dataset = above15k.sample(frac=0.8, random_state=0)
a15k_test_dataset = above15k.drop(a15k_train_dataset.index)

rem_train_dataset = remaining.sample(frac=0.8, random_state=0)
rem_test_dataset = remaining.drop(rem_train_dataset.index)

a15k_train_dataset.to_csv("lr_nn_data/above15k_train.csv")
a15k_test_dataset.to_csv("lr_nn_data/above15k_test.csv")

full_test = pd.concat([a15k_test_dataset, rem_test_dataset], ignore_index=True)
full_train = pd.concat([a15k_train_dataset, rem_train_dataset], ignore_index=True)

full_test.to_csv("lr_nn_data/full_test.csv")
full_train.to_csv("lr_nn_data/full_train.csv")